In [2]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
import os
import sys

import clip

import importlib

import contextlib

import json
import os
import random


sys.path.insert(0, os.path.abspath('../..')) 
sys.path.insert(0, os.path.abspath('../../..'))

import numpy as np
import torch 
from collections import defaultdict
from utils import get_default_path
from utils import Stopwatch
import time
import datetime


import ai.stabledisco as sd
import ai.torchmodules as torchmodules
import ai.torchmodules.data as torchdata
import ai.torchmodules.utils as torchutils
import ai.stabledisco.utils as sdutils
import ai.stabledisco.data as sddata
import ai.stabledisco.constants as sdconsts
import ai.torchmodules.pruning as torchprune
import torch
import torch.nn as nn
import pandas as pd

from clip.clip import _tokenizer as clip_tokenizer
import ai.stabledisco.decoderpipeline as decoderpipeline
  
tmp_dir =  "/home/ubuntu/tmp/"
tokens_path = os.path.join(tmp_dir, "text_tokens.pk")

In [3]:
text_tokens = torch.load(tokens_path)

In [4]:
if len(text_tokens.shape) > 1:
    text_tokens.view(1, text_tokens.size(0))
flipped = torch.flip(text_tokens, dims=(1,))
end_idx_arg = torch.argwhere(flipped == sdconsts.eot_token)[:,1]
for idx in range(flipped.size(0)):
    flipped[idx] = torch.cat((flipped[idx,end_idx_arg[idx]:], flipped[idx,:end_idx_arg[idx]]))
return flipped


In [11]:
rev_tokens_path = os.path.join(tmp_dir, "rev_text_tokens.pk")
text_tokens = torch.save(flipped, rev_tokens_path)